In [127]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from cmh import CMH

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [37]:
# Read CSV
df_1 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_1_choices.csv')
df_2 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_2_choices.csv')
df_3 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_3_choices.csv')
df_4 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_4_choices.csv')
df_5 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_5_choices.csv')
df_6 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_6_choices.csv')
df_7 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_7_choices.csv')
df_8 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_8_choices.csv')
df_9 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_9_choices.csv')
df_10 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_10_choices.csv')
df_11 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_11_choices.csv')
df_12 = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/switch_results/patient_12_choices.csv')

In [38]:
# Filter
df_1 = df_1[df_1['user_id'] != 'tim']
df_2 = df_2[df_2['user_id'] != 'tim']
df_3 = df_3[df_3['user_id'] != 'tim']
df_4 = df_4[df_4['user_id'] != 'tim']
df_5 = df_5[df_5['user_id'] != 'tim']
df_6 = df_6[df_6['user_id'] != 'tim']
df_7 = df_7[df_7['user_id'] != 'tim']
df_8 = df_8[df_8['user_id'] != 'tim']
df_9 = df_9[df_9['user_id'] != 'tim']
df_10 = df_10[df_10['user_id'] != 'tim']
df_11 = df_11[df_11['user_id'] != 'tim']
df_12 = df_12[df_12['user_id'] != 'tim']

In [39]:
# Add patient column
df_1['patient'] = 1
df_2['patient'] = 2
df_3['patient'] = 3
df_4['patient'] = 4
df_5['patient'] = 5
df_6['patient'] = 6
df_7['patient'] = 7
df_8['patient'] = 8
df_9['patient'] = 9
df_10['patient'] = 10
df_11['patient'] = 11
df_12['patient'] = 12

In [40]:
# Concatenate the DataFrames
dfs = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12]
all_choices = pd.concat(dfs, ignore_index=True)

In [41]:
# Order of seeing CDSS (1) or SOC (2)
# a and b are opposites of each other
# a = 122121122112
# b = 211212211221
a_list = [1,2,2,1,2,1,1,2,2,1,1,2]
b_list = [2,1,1,2,1,2,2,1,1,2,2,1]

In [42]:
# Define if see cdss or soc
def view_fun(row):
    patient = row['patient']
    user_archetype = row['user_archetype']
    if user_archetype == 'a':
        view = a_list[patient-1]
    elif user_archetype == 'b':
        view = b_list[patient-1]
    if view == 1:
        return 'cdss'
    elif view == 2:
        return 'soc'

all_choices['view'] = all_choices.apply(lambda row: view_fun(row), axis=1)

In [43]:
all_choices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         384 non-null    object
 1   user_archetype  384 non-null    object
 2   switch_choice   384 non-null    object
 3   explanation     377 non-null    object
 4   patient         384 non-null    int64 
 5   view            384 non-null    object
dtypes: int64(1), object(5)
memory usage: 18.1+ KB


In [44]:
all_choices

,user_id,user_archetype,switch_choice,explanation,patient,view
0,0,a,dont_switch,"temps increasing, systolics dropping, still on...",1,cdss
1,1,b,dont_switch,The respiratory rates are increasing.,1,soc
2,2,a,dont_switch,30 hours is not long enough to assess for resp...,1,cdss
3,3,b,dont_switch,"Raised inflammatory markers (raised CRP, RR>20",1,soc
4,4,a,dont_switch,Based on CURB score the patient is at least CU...,1,cdss
...,...,...,...,...,...,...
379,27,b,dont_switch,Unless NGg tube in place GCSs may cause proble...,12,cdss
380,28,a,switch,Stable for 24 hours no clear red flags,12,soc
381,29,b,dont_switch,GCS low; low grade fevers,12,cdss
382,30,a,dont_switch,"Low GCS., I would want more information about ...",12,soc


In [45]:
# Add demographics 
demographics = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/demographic_results/demographics_processed.csv')
all_choices['user_id'] = all_choices['user_id'].astype(int)
all_choices = pd.merge(all_choices, demographics[['user_id', 'user_archetype', 'age_new', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new']])

In [46]:
all_choices

,user_id,user_archetype,switch_choice,explanation,patient,view,age_new,sex_new,medical_speciality_new,grade_new,ai_familiarity_new
0,0,a,dont_switch,"temps increasing, systolics dropping, still on...",1,cdss,30s,Male,Pharmacist,Other,NaN
1,0,a,switch,"apyrexial, HR returned to normal",2,soc,30s,Male,Pharmacist,Other,NaN
2,0,a,dont_switch,"temp and HR increasing, GCS dropped in last 24...",3,soc,30s,Male,Pharmacist,Other,NaN
3,0,a,switch,"temps and other vitals all ok, and have been s...",4,cdss,30s,Male,Pharmacist,Other,NaN
4,0,a,switch,all vitals in range,5,soc,30s,Male,Pharmacist,Other,NaN
...,...,...,...,...,...,...,...,...,...,...,...
379,31,b,switch,"Switch, provided diagnosis non-complex and inv...",8,cdss,30s,Male,Infectious Diseases,Other,Moderately familiar
380,31,b,switch,Obs normalised. Long duration of IV abx already.,9,cdss,30s,Male,Infectious Diseases,Other,Moderately familiar
381,31,b,dont_switch,"Hypotensive, seemingly new. Would need additio...",10,soc,30s,Male,Infectious Diseases,Other,Moderately familiar
382,31,b,dont_switch,Recent period of significant hypotension. This...,11,soc,30s,Male,Infectious Diseases,Other,Moderately familiar


In [47]:
all_choices['view'].value_counts()
all_choices['switch_choice'].value_counts()
all_choices['patient'].value_counts()

view
cdss    192
soc     192
Name: count, dtype: int64

switch_choice
switch         237
dont_switch    147
Name: count, dtype: int64

patient
1     32
2     32
3     32
4     32
5     32
6     32
7     32
8     32
9     32
10    32
11    32
12    32
Name: count, dtype: int64

In [71]:
all_choices.groupby(['patient', 'view'])['switch_choice'].value_counts()

patient  view  switch_choice
1        cdss  dont_switch      16
         soc   dont_switch      16
2        cdss  switch           16
         soc   switch           14
               dont_switch       2
3        cdss  dont_switch      13
               switch            3
         soc   dont_switch      11
               switch            5
4        cdss  switch           14
               dont_switch       2
         soc   switch           16
5        cdss  switch           16
         soc   switch           14
               dont_switch       2
6        cdss  switch           12
               dont_switch       4
         soc   switch           16
7        cdss  dont_switch      12
               switch            4
         soc   switch           12
               dont_switch       4
8        cdss  switch           10
               dont_switch       6
         soc   switch            8
               dont_switch       8
9        cdss  switch           14
               dont_switch

# Chi-squared

In [50]:
# Contingency table
contingency_table = pd.crosstab(all_choices['view'], all_choices['switch_choice'])

# Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square: {chi2}, p-value: {p}")

Chi-square: 3.571170240247998, p-value: 0.05879086619199658


In [139]:
# Subset the data for the specific patient
subset_df = all_choices[all_choices['patient'] == 7]

# Contingency table
contingency_table = pd.crosstab(subset_df['view'], subset_df['switch_choice'])

In [54]:
# Loop through each unique patient
for patient in all_choices['patient'].unique():
    # Subset the data for the specific patient
    subset_df = all_choices[all_choices['patient'] == patient]
    
    # Contingency table
    contingency_table = pd.crosstab(subset_df['view'], subset_df['switch_choice'])
    
    # Chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    print(f"Patient: {patient}")
    print(f"Chi-square: {chi2}, p-value: {p}")
    if p < 0.05:
        print(f"  -> The differences in are statistically significant.\n")
    else:
        print(f"  -> The differences in are ***NOT*** statistically significant.\n")
    print("-" * 30)

Patient: 1
Chi-square: 0.0, p-value: 1.0
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Patient: 2
Chi-square: 0.5333333333333333, p-value: 0.4652088184521417
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Patient: 3
Chi-square: 0.16666666666666666, p-value: 0.6830913983096086
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Patient: 4
Chi-square: 0.5333333333333333, p-value: 0.4652088184521417
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Patient: 5
Chi-square: 0.5333333333333333, p-value: 0.4652088184521417
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Patient: 6
Chi-square: 2.571428571428571, p-value: 0.10880943004054607
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Patient: 7
Chi-square

In [131]:
all_choices.columns

Index(['user_id', 'user_archetype', 'switch_choice', 'explanation', 'patient',
       'view', 'age_new', 'sex_new', 'medical_speciality_new', 'grade_new',
       'ai_familiarity_new', 'view_binary', 'switch_choice_binary'],
      dtype='object')

In [134]:
# Loop through each demographic
demographic_list = ['age_new', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new']
for demographic in demographic_list:
    # Contingency table
    contingency_table = pd.crosstab(all_choices[demographic], all_choices['switch_choice'])
    
    # Chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    print(f"Demographic: {demographic}")
    print(f"Chi-square: {chi2}, p-value: {p}")
    if p < 0.05:
        print(f"  -> The differences in are statistically significant.\n")
    else:
        print(f"  -> The differences in are ***NOT*** statistically significant.\n")
    print("-" * 30)

Demographic: age_new
Chi-square: 4.988026059838823, p-value: 0.2885283044057984
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Demographic: sex_new
Chi-square: 0.0, p-value: 1.0
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Demographic: medical_speciality_new
Chi-square: 6.624701157793382, p-value: 0.08487222049555458
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Demographic: grade_new
Chi-square: 1.2062791251631344, p-value: 0.27207028561313146
  -> The differences in are ***NOT*** statistically significant.

------------------------------
Demographic: ai_familiarity_new
Chi-square: 6.632804232804233, p-value: 0.08456962624264629
  -> The differences in are ***NOT*** statistically significant.

------------------------------


In [138]:
# Loop through each unique patient
for patient in all_choices['patient'].unique():
    # Loop through each demographic
    demographic_list = ['age_new', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new']
    for demographic in demographic_list:
        # Contingency table
        contingency_table = pd.crosstab(all_choices[demographic], all_choices['switch_choice'])
        
        # Chi-square test
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        if p < 0.05:
            print(f"Patient: {patient}")
            print(f"Demographic: {demographic}")
            print(f"Chi-square: {chi2}, p-value: {p}")
            print(f"  -> The differences in are statistically significant.\n")
        #else:
        #    print(f"  -> The differences in are ***NOT*** statistically significant.\n")
        #print("-" * 30)
print('Done')

Done


# Others

In [ ]:
# Cant get these to work atm

In [ ]:
# Define contingency table
contingency_table = pd.crosstab(all_choices['switch_choice'], all_choices['view'])

# Fit a log-linear model
exog, endog = sm.add_constant(all_choices['view']), all_choices['switch_choice']
mod = sm.GLM(endog, exog, family=sm.families.Poisson ,link=sm.families.links.log)
res = mod.fit()

# Summary of the results
print(result.summary())

In [ ]:
# Perform Cochran-Mantel-Haenszel Test
result = CMH(all_choices, 'switch_choice', 'age_new', stratifier='view')
display(result)

# LR

In [64]:
# Create a mapping dictionary
view_mapping = {'cdss': 1, 'soc': 0, '1': 1, '0': 0}
switch_choice_mapping = {'switch': 1, 'dont_switch': 0, '1': 1, '0': 0}

# Map categorical values to numeric
all_choices['view_binary'] = all_choices['view'].map(view_mapping).astype('int')
all_choices['switch_choice_binary'] = all_choices['switch_choice'].map(switch_choice_mapping).astype('int')

In [65]:
# Logistic regression model
X = all_choices['view_binary']
y = all_choices['switch_choice_binary']

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fitting the model
logit_model = sm.Logit(y, X).fit(disp=0)

# Model summary
print(logit_model.summary())

                            Logit Regression Results                            
Dep. Variable:     switch_choice_binary   No. Observations:                  384
Model:                            Logit   Df Residuals:                      382
Method:                             MLE   Df Model:                            1
Date:                  Wed, 10 Jul 2024   Pseudo R-squ.:                0.007803
Time:                          18:05:25   Log-Likelihood:                -253.53
converged:                         True   LL-Null:                       -255.52
Covariance Type:              nonrobust   LLR p-value:                   0.04584
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.6931      0.153      4.528      0.000       0.393       0.993
view_binary    -0.4206      0.211     -1.990      0.047      -0.835      -0.006


LLR p-value:
LLR p-value: 0.04584 - This is the p-value for the likelihood ratio test, which tests whether your model with predictors is significantly better than a model with no predictors. A p-value less than 0.05 indicates that your model is statistically significant.

Interpretation of Coefficient for view_binary
Coefficient (coef): -0.4206
The negative coefficient for view_binary means that an increase in the value of view_binary is associated with a decrease in the log odds of the outcome switch_choice_binary being 1.
To interpret this in terms of odds ratio:
Odds Ratio: exp(coef)=exp(−0.4206)≈0.657
The odds of switch_choice_binary being 1 are about 65.7% of the odds when view_binary is 0. In other words, if view_binary changes from 0 to 1, the odds of switch_choice_binary being 1 decrease by approximately 34.3%.
Significance (P>|z|): 0.047

The p-value is less than 0.05, which indicates that the effect of view_binary on switch_choice_binary is statistically significant.
Summary
Intercept: Significant, but its practical interpretation depends on the context of switch_choice_binary.
view_binary: Has a statistically significant negative effect on switch_choice_binary, suggesting that when view_binary increases, the probability of switch_choice_binary being 1 decreases.

In [76]:
# Loop through each unique patient
for patient in all_choices['patient'].unique():
    # Subset the data for the specific patient
    subset_df = all_choices[all_choices['patient'] == patient]

    # Logistic regression model
    X = subset_df['view_binary']
    y = subset_df['switch_choice_binary']

    # Adding a constant to the model (intercept)
    X = sm.add_constant(X)

    # Fitting the model
    logit_model = sm.Logit(y, X).fit(disp=0)

    # Model summary
    print(f"Patient: {patient}")
    print(logit_model.summary())
    print("-" * 30)

Patient: 1
                            Logit Regression Results                            
Dep. Variable:     switch_choice_binary   No. Observations:                   32
Model:                            Logit   Df Residuals:                       30
Method:                             MLE   Df Model:                            1
Date:                  Wed, 10 Jul 2024   Pseudo R-squ.:                     inf
Time:                          18:34:51   Log-Likelihood:            -2.4486e-07
converged:                        False   LL-Null:                        0.0000
Covariance Type:              nonrobust   LLR p-value:                     1.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const         -17.9952   2020.883     -0.009      0.993   -3978.852    3942.862
view_binary   -26.7135   1.28e+09  -2.09e-08      1.000    -2.5e+09     2.5e+09

Complete Separation:

/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Lik

Those that are statistically significant

6 - pvalue 0.01 - Odds Ratio: exp(coef)=exp(-19.6972)≈0. Model failed to converge. 


7 - pvalue 0.003 - Odds Ratio: exp(coef)=exp(-19.6972)≈0.11. The odds of switch_choice_binary being 1 are about 11% of the odds when view_binary is 0. In other words, if view_binary changes from 0 to 1, the odds of switch_choice_binary being 1 decrease by approximately 89%. Has a statistically significant negative effect on switch_choice_binary, suggesting that when view_binary increases, the probability of switch_choice_binary being 1 decreases. Good 7 CDSS says dont switch! How is this possible when decisions are the same? 

In [122]:
# Categorical Variables
all_choices_interaction = all_choices.copy()
all_choices_interaction.drop(columns=['view_binary'], inplace=True)
# One-hot encoding categorical variables
df_encoded = pd.get_dummies(all_choices_interaction, columns=['view', 'age_new', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new'], drop_first=True)

view_columns = ['view_soc']
demographic_columns = df_encoded.iloc[:,8:].columns.to_list()

# Convert to 0 and 1
df_encoded[demographic_columns] = df_encoded[demographic_columns].astype(int)
df_encoded[view_columns] = df_encoded[view_columns].astype(int)

# Fit logistic regression model by patient
results = {}
for patient in df_encoded['patient'].unique():
    # Subset the data for the specific patient
    subset_df = df_encoded[df_encoded['patient'] == patient]
    
    # Drop rows with missing values
    subset_df = subset_df.dropna(subset=['switch_choice'])
    
    # Check if there are enough rows to fit the model
    if len(subset_df) < 2:
        print(f"Patient: {patient}")
        print("Not enough data to fit the model.")
        print("-" * 30)
        continue
    
    # Logistic regression model
    X = subset_df[view_columns + demographic_columns]
    y = subset_df['switch_choice_binary']
    
    # Ensure X and y are in the correct format
    X = sm.add_constant(X)  # Add intercept
    
    # Fitting the model
    try:
        logit_model = sm.Logit(y, X).fit(disp=0)
        results[patient] = logit_model
        print(f"Patient: {patient}")
        print(logit_model.summary())
    except Exception as e:
        print(f"Error: {e}")
    
    print("-" * 30)

Patient: 1
                            Logit Regression Results                            
Dep. Variable:     switch_choice_binary   No. Observations:                   32
Model:                            Logit   Df Residuals:                       19
Method:                             MLE   Df Model:                           12
Date:                  Wed, 10 Jul 2024   Pseudo R-squ.:                     inf
Time:                          19:21:28   Log-Likelihood:            -1.5321e-13
converged:                        False   LL-Null:                        0.0000
Covariance Type:              nonrobust   LLR p-value:                     1.000
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                  -28.0814   3.31e+07  -8.47e-07      1.000   -6.49e+07    6.49e+07
view_soc                  

/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  wa

In [118]:
# Interaction Terms with Categorical Variables
all_choices_interaction = all_choices.copy()
all_choices_interaction.drop(columns=['view_binary'], inplace=True)
# One-hot encoding categorical variables
df_encoded = pd.get_dummies(all_choices_interaction, columns=['view', 'age_new', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new'], drop_first=True)

view_columns = ['view_soc']
demographic_columns = df_encoded.iloc[:,8:].columns.to_list()
# Generate interaction terms
interaction_terms = []
for view_col in view_columns:
    for dem_col in demographic_columns:
        interaction_term = f'{view_col}:{dem_col}'
        df_encoded[interaction_term] = df_encoded[view_col] * df_encoded[dem_col]
        interaction_terms.append(interaction_term)

# Convert to 0 and 1
df_encoded[demographic_columns] = df_encoded[demographic_columns].astype(int)
df_encoded[interaction_terms] = df_encoded[interaction_terms].astype(int)
df_encoded[view_columns] = df_encoded[view_columns].astype(int)

# Fit logistic regression model by patient
results = {}
for patient in df_encoded['patient'].unique():
    # Subset the data for the specific patient
    subset_df = df_encoded[df_encoded['patient'] == patient]
    
    # Drop rows with missing values
    subset_df = subset_df.dropna(subset=['switch_choice'])
    
    # Check if there are enough rows to fit the model
    if len(subset_df) < 2:
        print(f"Patient: {patient}")
        print("Not enough data to fit the model.")
        print("-" * 30)
        continue
    
    # Logistic regression model
    X = subset_df[view_columns + demographic_columns + interaction_terms]
    y = subset_df['switch_choice_binary']
    
    # Ensure X and y are in the correct format
    X = sm.add_constant(X)  # Add intercept
    
    # Fitting the model
    try:
        logit_model = sm.Logit(y, X).fit(disp=0)
        results[patient] = logit_model
        print(f"Patient: {patient}")
        print(logit_model.summary())
    except Exception as e:
        print(f"Error: {e}")
    
    print("-" * 30)

# Print results for each patient
for patient, result in results.items():
    print(f"Patient: {patient}")
    print(result.summary())
    print("-" * 30)

Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------
Error: Singular matrix
------------------------------


/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/wb1115/anaconda3/envs/cdss/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  wa